In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount = True)
dataset_path = 'gdrive/My Drive/Deep Learning/sentiment analysis/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!pip install pytreebank

  Created wheel for pytreebank: filename=pytreebank-0.2.7-cp36-none-any.whl size=37072 sha256=a7fdfa82cd065cf1c2fe65cbdb3279cbfddebf50f5193f826dcd9a5fbf4be4dd
  Stored in directory: /root/.cache/pip/wheels/e0/b6/91/e9edcdbf464f623628d5c3aa9de28888c726e270b9a29f2368
Successfully built pytreebank


In [ ]:
import pytreebank

In [ ]:
import sys
import os

out_path = os.path.join(dataset_path,sys.path[0], 'sst_{}.txt')
print(out_path)
dataset = pytreebank.load_sst('./raw_data')
# Store train, dev and test in separate files
for category in ['train', 'test', 'dev']:
    with open(out_path.format(category), 'w') as outfile:
        for item in dataset[category]:
            outfile.write("{}\t{}\n".format(
                item.to_labeled_lines()[0][1],
                item.to_labeled_lines()[0][0] + 1,
            ))
# Print the length of the training set
print(len(dataset['train']))

gdrive/My Drive/Deep Learning/sentiment analysis/sst_{}.txt
8544


In [ ]:
import pandas as pd
# Read train data
for file in ['sst_train','sst_test','sst_dev']:
  print(file)
  df = pd.read_csv(dataset_path+file+'.txt', sep='\t', header=None, names=['text', 'truth'])
  #df['truth'] = df['truth'].str.replace('__label__', '')
  df['truth'] = df['truth'].astype(int).astype('category')
  print(df.shape[0])
  print(df.head())
  df.to_csv(dataset_path + file+'.csv',index = False)


sst_train
8544
                                                text truth
0  The Rock is destined to be the 21st Century 's...     4
1  The gorgeously elaborate continuation of `` Th...     5
2  Singer/composer Bryan Adams contributes a slew...     4
3  You 'd think by now America would have had eno...     3
4               Yet the act is still charming here .     4
sst_test
2210
                                                text truth
0                     Effective but too-tepid biopic     3
1  If you sometimes like to go to the movies to h...     4
2  Emerges as something rare , an issue movie tha...     5
3  The film provides some great insight into the ...     3
4  Offers that rare combination of entertainment ...     5
sst_dev
1101
                                                text truth
0  It 's a lovely film with lovely performances b...     4
1  No one goes unindicted here , which is probabl...     3
2  And if you 're not nearly moved to tears by a ...     4
3             

In [ ]:
import torch
from torchtext import data

SEED = 1234
import pandas as pd
import numpy as np
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext

import nltk

import random
from sklearn.metrics import classification_report

#import pyprind
%matplotlib inline  

In [ ]:
import spacy
spacy_en = spacy.load('en')

In [ ]:
is_cuda = torch.cuda.is_available()
print("Cuda Status on system is {}".format(is_cuda))

Cuda Status on system is True


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import string

def tokenizer(text):
  tokens = [tok for tok in nltk.word_tokenize(text)]
  #lower_token = [w.lower() for w in tokens]
  #table = str.maketrans('','',string.punctuation)
  #stripped = [w.translate(table) for w in lower_token]
  #alpha_words = [w for w in tokens if w.isalpha()]
  #stop_words = set(stopwords.words('english'))
  #final_words = [w for w in alpha_words if w not in stop_words]
  #lemmatized_words = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
  return tokens

In [ ]:
TEXT = data.Field(sequential=True, tokenize=tokenizer, include_lengths=True)
LABEL = data.LabelField(dtype=torch.long, sequential=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_data, valid_data, test_data = data.TabularDataset.splits(
    path=dataset_path, train="sst_train.csv", 
    validation="sst_dev.csv", test="sst_test.csv",format="csv", skip_header=True, 
    fields=[('text', TEXT), ('truth', LABEL)]
)

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of valid examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 8544
Number of valid examples: 1101
Number of testing examples: 2210


In [ ]:
TEXT.build_vocab(train_data, vectors=torchtext.vocab.Vectors(dataset_path+"glove.840B.300d.txt"), 
                 max_size=25000,unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

100%|█████████▉| 2195129/2196017 [04:01<00:00, 9236.60it/s]

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 18272
Unique tokens in LABEL vocabulary: 5


In [ ]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# keep in mind the sort_key option 
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), sort_key=lambda x: len(x.text),
    batch_size=BATCH_SIZE,sort_within_batch = True,
    device=device)
'''train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)'''
LABEL.vocab.freqs

Counter({'1': 1092, '2': 2218, '3': 1624, '4': 2322, '5': 1288})

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, bidirectional,n_layers,dropout,pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        #self.embedding.weight.requires_grad = False
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional)
        if bidirectional == True:
          self.fc = nn.Linear(hidden_dim*2, output_dim)
        else:
          self.fc = nn.Linear(hidden_dim, output_dim)
        self.ui = nn.Linear(2*hidden_dim, 100)
        self.ui1 = nn.Linear(2*hidden_dim, 100)
        self.v = nn.Parameter(torch.zeros(100))
        #self.uw = nn.Parameter(torch.randn(100))
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.Softmax(dim=-1)
        self.bidirectional_used = bidirectional
        self.query = nn.Parameter(torch.zeros(1000))

    def forward(self, x, x_len, hidden):

        #x = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(x))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, x_len)
        #embedded = [sent len, batch size, emb dim]
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        shape = output.shape[0]
        batch = output.shape[1]
        
        if self.bidirectional_used == True:
          hid = 2*self.hidden_dim
        else:
          hid = self.hidden_dim

        
        u_it = self.ui(output)
        u_it1 = self.ui1(self.query)
        u_sum = u_it + u_it1
        weights = torch.softmax((u_sum).matmul(self.v), dim=0).unsqueeze(2)
        weights3D = torch.repeat_interleave(weights, 1000, axis=2)
        
        sent = torch.sum(weights3D*output, dim=0) # B X HdD*2
        #print(sent.shape)
        if self.bidirectional_used == True:
          layer_list = [-1, -2]
        else:
          layer_list = [-1]
        hidden = torch.cat(tuple([hidden[i, :, :] for i in layer_list]), dim=1)
        #print(hidden.shape)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        #assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        #self.fc = nn.Linear(shape,self.output_dim)
        #out = self.fc(output)
        out = self.fc(sent)
        return out
    def initHidden(self):
        hidden = torch.zeros(1, self.hidden_dim)
        #query = torch.zeros(1,self.hidden_dim)
        #torch.nn.init.xavier_normal_(query)
        torch.nn.init.xavier_normal_(hidden)
        return hidden #torch.zeros(1, self.hidden_dim)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 500
OUTPUT_DIM = 5
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.6
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM,BIDIRECTIONAL,N_LAYERS,DROPOUT,PAD_IDX) 
'''N_LAYERS,BIDIRECTIONAL,DROPOUT)'''

'N_LAYERS,BIDIRECTIONAL,DROPOUT)'

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 14,903,905 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([18272, 300])


In [ ]:
#model.embedding.weight.data = pretrained_embeddings.cuda()
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0120,  0.2075, -0.1258,  ...,  0.1387, -0.3605, -0.0350],
        ...,
        [ 0.0495, -0.2737, -0.2819,  ..., -0.2686,  0.5445,  0.1999],
        [ 0.8430, -0.0559, -0.0837,  ...,  0.9208, -0.2708, -0.4322],
        [ 0.4218,  0.2891,  0.6224,  ..., -0.0994, -0.3216, -0.2066]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0120,  0.2075, -0.1258,  ...,  0.1387, -0.3605, -0.0350],
        ...,
        [ 0.0495, -0.2737, -0.2819,  ..., -0.2686,  0.5445,  0.1999],
        [ 0.8430, -0.0559, -0.0837,  ...,  0.9208, -0.2708, -0.4322],
        [ 0.4218,  0.2891,  0.6224,  ..., -0.0994, -0.3216, -0.2066]])


In [ ]:
#class_weights = torch.tensor([1.0, 15.0]).cuda()
model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-4)#,weight_decay=0.001)
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds1, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    preds, ind= torch.max(F.softmax(preds1),1)
    correct = (ind == y).float()
    acc = correct.sum()/float(len(correct))
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        hidden = model.initHidden()
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths,hidden).squeeze(1)
        
        loss = criterion(predictions, batch.truth)
        
        acc = binary_accuracy(predictions, batch.truth)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        #bar = pyprind.ProgBar(len(iterator), bar_char='█')
        for batch in iterator:
            hidden = model.initHidden()
            text, text_lengths = batch.text
            predictions = model(text, text_lengths,hidden).squeeze(1)
            
            loss = criterion(predictions, batch.truth)
            
            acc = binary_accuracy(predictions, batch.truth)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            #bar.update()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 10

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


| Epoch: 01 | Train Loss: 1.461 | Train Acc: 35.60% | Val. Loss: 1.338 | Val. Acc: 40.52% |
| Epoch: 02 | Train Loss: 1.343 | Train Acc: 40.71% | Val. Loss: 1.307 | Val. Acc: 41.55% |
| Epoch: 03 | Train Loss: 1.310 | Train Acc: 42.18% | Val. Loss: 1.313 | Val. Acc: 42.66% |
| Epoch: 04 | Train Loss: 1.276 | Train Acc: 43.60% | Val. Loss: 1.291 | Val. Acc: 42.58% |
| Epoch: 05 | Train Loss: 1.256 | Train Acc: 45.10% | Val. Loss: 1.269 | Val. Acc: 45.03% |
| Epoch: 06 | Train Loss: 1.230 | Train Acc: 46.34% | Val. Loss: 1.274 | Val. Acc: 44.85% |
| Epoch: 07 | Train Loss: 1.203 | Train Acc: 48.27% | Val. Loss: 1.257 | Val. Acc: 44.90% |
| Epoch: 08 | Train Loss: 1.182 | Train Acc: 49.47% | Val. Loss: 1.244 | Val. Acc: 45.92% |
| Epoch: 09 | Train Loss: 1.164 | Train Acc: 49.81% | Val. Loss: 1.248 | Val. Acc: 45.43% |
| Epoch: 10 | Train Loss: 1.138 | Train Acc: 50.50% | Val. Loss: 1.232 | Val. Acc: 46.81% |


In [ ]:
test_add_notan = []

In [ ]:

#model.load_state_dict(torch.load(dataset_path + 'tut1-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
test_add_notan.append(test_acc*100)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


Test Loss: 1.196 | Test Acc: 47.59%


In [ ]:
test_add_notan

[46.919642857142854, 48.16964285714286, 46.517857142857146, 47.58928571428571]

In [ ]:
sum(test_norm_tanh)/len(test_norm_tanh)

47.27678571428571

In [ ]:
sum(test_1)/len(test_1)

47.69642857142857

In [ ]:
sum(test)/len(test)

47.84821428571429

In [ ]:
test_tanh

[46.964285714285715,
 47.767857142857146,
 48.16964285714286,
 47.5,
 47.00892857142857,
 46.60714285714286]

In [ ]:
sum(test_tanh[:-1])/len(test_tanh[:-1])

47.482142857142854

In [ ]:
sum(test_add)/len(test_add)

47.481142857142856

In [ ]:
sum(test_add_notan)/len(test_add_notan)

47.29910714285714